In [ ]:
%conda install pandas gdal scikit-learn scikit-image scipy matplotlib seaborn

Retrieving notices: done
Channels:
 - defaults
Platform: linux-64
Solving environment: done

# All requested packages already installed.


Note: you may need to restart the kernel to use updated packages.


In [14]:
import os
from pathlib import Path
import rasterio
import numpy as np
import os
import pandas as pd
import shutil

np.seterr(divide='ignore', invalid='ignore')
os.chdir("/home/me/workspace/det_remota/trabalho_final")

In [15]:
RAW_FILES_PATH = Path("data/sentinel2/merged")

def parse_file_path(path:Path):
    file_info = str(path).replace(f"{RAW_FILES_PATH}/", '')
    file_info = file_info.replace('.tiff', '')
    layer, year, filename = file_info.split('/')

    
    return {
        'layer' :layer,
        'year' :year,
        'month' :filename.split('_')[-1],
        'path': path,
    }

files_df = pd.DataFrame(
    [
        parse_file_path(path)
        for path in RAW_FILES_PATH.glob('**/*.tiff')
    ]
)
files_df.groupby(['year', 'month']).path.count()

year  month
2017  08       6
2018  08       6
Name: path, dtype: int64

In [ ]:
import shutil

AGGREGATED_PATH = Path('data/sentinel2/aggregated')
AGGREGATED_PATH.mkdir(parents=True, exist_ok=True)
NORMALIZED_PATH = Path('data/sentinel2/normalized')
NORMALIZED_PATH.mkdir(parents=True, exist_ok=True)

def apply_constrast(channel_data):
    min_max = (channel_data - np.min(channel_data)) / (np.max(channel_data) - np.min(channel_data))
    return 255 * min_max

def aggregate_tiff(year, month, path: list[Path]):
    

    print(f"Pre-Processing raster for {year}/{month}")

    imgs_dict = {}
    for _path in path:
        if 'MOISTURE' in str(_path):
            img_index = "ndmi"
        if 'MSAVI2' in str(_path):
            img_index = "msavi2"
        if 'NDWI' in str(_path):
            img_index = "ndwi"
        if 'VEGETATION_INDEX' in str(_path):
            img_index = "ndvi"
        if 'FALSE-COLOR' in str(_path):
            img_index = "false_color"
        if 'TRUE_COLOR' in str(_path):
            img_index = "true_color"
        imgs_dict[img_index] = rasterio.open(_path)
    
    channels_values = [
        # True Color first
        imgs_dict["true_color"].read(1),
        imgs_dict["true_color"].read(2),
        imgs_dict["true_color"].read(3),
        # False Color second
        imgs_dict["false_color"].read(1),
        imgs_dict["false_color"].read(2),
        imgs_dict["false_color"].read(3),
        # Indexes in the tails
        imgs_dict["ndmi"].read(1),
        imgs_dict["msavi2"].read(1),
        imgs_dict["ndwi"].read(1),
        imgs_dict["ndvi"].read(1),
        # One mask for NODATA
        imgs_dict["ndvi"].read(2),
    ]

    profile = imgs_dict["ndvi"].profile.copy()
    profile.update(count=len(channels_values))

    agg_out_path = AGGREGATED_PATH.joinpath(f'{year}_{month.zfill(2)}.tiff')
    with rasterio.open(agg_out_path, "w", **profile) as out_tiff:
        for idx, values in enumerate(channels_values):
            out_tiff.write_band(idx+1, values)
    return True

In [20]:
OUT_CHANNEL_NAMES = [
    "TRUE_COLOR_1",
    "TRUE_COLOR_2",
    "TRUE_COLOR_3",
    "FALSE_COLOR_1",
    "FALSE_COLOR_2",
    "FALSE_COLOR_3",
    "NDMI",
    "MSAVI2",
    "NDWI",
    "NDVI",
    "NODATA_MASK",
]

if True:
    if AGGREGATED_PATH.exists():
        shutil.rmtree(AGGREGATED_PATH)
    AGGREGATED_PATH.mkdir(parents=True)
    if NORMALIZED_PATH.exists():
        shutil.rmtree(NORMALIZED_PATH)
    NORMALIZED_PATH.mkdir(parents=True)

if True:
    pd.DataFrame({
        'Band': range(1, len(OUT_CHANNEL_NAMES)+1),
        'Description': OUT_CHANNEL_NAMES
    }).to_csv(AGGREGATED_PATH.joinpath('bands.csv'), index=False)
    pd.DataFrame({
        'Band': range(1, len(OUT_CHANNEL_NAMES)+1),
        'Description': OUT_CHANNEL_NAMES
    }).to_csv(NORMALIZED_PATH.joinpath('bands.csv'), index=False)

    files_df\
        .groupby(['year', 'month'])\
        .path.unique().reset_index()\
        .apply(lambda x: aggregate_tiff(**x), axis=1)

Pre-Processing raster for 2017/08
Pre-Processing raster for 2018/08
